In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


한국어 감정 정보가 포함된 데이터셋

In [26]:
keti_single=pd.read_excel('/content/drive/My Drive/SookLog/한국어_단발성_대화_데이터셋.xlsx')
keti_single.head()

keti_single=keti_single.iloc[:,:2]
keti_single.head()

keti_single.rename(columns={'Sentence':"발화",'Emotion':'감정'},inplace=True)
keti_single[keti_single["감정"]=="놀람"].sample(50)

# "놀람"에 해당하는 텍스트들이 "중립","혐오","당황" 등을 넘나들기 때문에 "놀람"에 해당하는 데이터들을 삭제한다.

keti_single=keti_single[keti_single["감정"]!="놀람"]
keti_single["감정"]=np.where(keti_single["감정"].str.match("공포"),"불안",keti_single["감정"])

감성 대화 데이터셋

In [ ]:
fin_t=pd.read_excel('/content/drive/My Drive/SookLog/감성대화말뭉치(최종데이터)_Training.xlsx')
fin_v=pd.read_excel('/content/drive/My Drive/SookLog/감성대화말뭉치(최종데이터)_Validation.xlsx')

In [ ]:
fin_t=fin_t.fillna("")
fin_v=fin_v.fillna("")

In [ ]:
print(keti_single.columns)

Index(['발화', '감정'], dtype='object')


In [ ]:
# 컬럼간 str 합치기

fin_t["사람문장"]=fin_t["사람문장1"].astype(str)+fin_t["사람문장2"].astype(str)+fin_t["사람문장3"]
fin_v["사람문장"]=fin_v["사람문장1"].astype(str)+fin_v["사람문장2"].astype(str)+fin_v["사람문장3"]

In [ ]:
df_concat=pd.concat([fin_t,fin_v])
final_df=df_concat[["사람문장","감정_대분류"]]
final_df=final_df.rename({"감정_대분류":"감정"},axis=1)

In [ ]:
#컬럼 내 데이터들의 공백(string) 제거

final_df["감정"]=final_df["감정"].apply(lambda x:x.strip())

final_df["감정"].value_counts()

,count
감정,
불안,10433
분노,10417
상처,10150
슬픔,10128
당황,9804
기쁨,7339


In [ ]:
#np.where(조건,True일때 값,False일때 값)

final_df["감정"]=np.where(final_df["감정"].str.match("상처"),"슬픔",final_df["감정"])
final_df["감정"]=np.where(final_df["감정"].str.match("기쁨"),"행복",final_df["감정"])

final_df=final_df.reset_index(drop=True)
final_df.rename({"사람문장":"발화"},axis=1,inplace=True)

연속적 대화 데이터셋

In [ ]:
keti_dialogue=pd.read_excel('/content/drive/My Drive/SookLog/한국어_연속적_대화_데이터셋.xlsx')

In [ ]:
keti_dialogue.head()

# 첫행을 컬럼으로 바꾸기
new_columns=keti_dialogue.iloc[0]
keti_dialogue=keti_dialogue.rename(columns=new_columns).iloc[:,1:3]
keti_dialogue.drop(index=0,axis=0,inplace=True)
keti_dialogue.head()

keti_dialogue["감정"].value_counts()

,count
감정,
중립,43786
놀람,4866
분노,3628
슬픔,1972
행복,1030
혐오,220
공포,98
ㅍ,12
분,4


In [ ]:
# 특정 감정들만 필터링
keti_dialogue = keti_dialogue[keti_dialogue["감정"].isin(["중립", "놀람", "분노", "슬픔", "행복", "혐오", "공포"])]

# 감정 Label 일정하게 맞추기 (감성대화 말뭉치와 통합을 위해)
keti_dialogue.loc[keti_dialogue["감정"].str.match("놀람"), "감정"] = "당황"
keti_dialogue.loc[keti_dialogue["감정"].str.match("공포"), "감정"] = "불안"

전체 대화 데이터셋 합치기


In [ ]:
real_final_df=pd.concat([final_df,keti_dialogue,keti_single])
real_final_df.reset_index(drop=True,inplace=True)
real_final_df

,발화,감정
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테...,분노
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,분노
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,분노
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,분노
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.상사인 나에게 먼...,분노
...,...,...
141598,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오
141599,재미가 없으니 망하지,혐오
141600,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오
141601,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오


In [ ]:
real_final_df["감정"].value_counts()

,count
감정,
중립,48616
슬픔,27517
분노,19710
불안,15901
행복,14406
당황,9804
혐오,5649


In [ ]:
from google.colab import files
real_final_df.to_csv("sentiment_dialogues.csv", index=False)
files.download("sentiment_dialogues.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>